In [31]:
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd
import requests
import QuantLib as ql
import numpy as np
import matplotlib.pyplot as plt
import json


with open("config.json", "r") as config_file:
    config = json.load(config_file)

api_key = config.get("api_key")
secret_key = config.get("secret_key")


In [32]:
NASDAQ = pd.read_csv('Indexes/NASDAQ.csv')
DOWJ = pd.read_csv('Indexes/DOWJ.csv')
SP = pd.read_csv('Indexes/S&P500.csv')

def clean_data(df):
    df = df[['Company', 'Symbol']]
    df = pd.DataFrame(df).dropna()
    return df

NASDAQ = clean_data(NASDAQ)
DOWJ = clean_data(DOWJ)
SP = clean_data(SP)


In [44]:

def get_price_change_and_rolling_avg(ticker: str):

    end_date = datetime.now()
    start_date = end_date - timedelta(days=24)
    
    data = yf.download(ticker, start=start_date, end=end_date, progress=False)
    
    data = data.sort_index()
    
    data['Price_Change'] = (data['Close'].diff() / data['Close'].shift(1)) * 100
    today_price_change = data['Price_Change'].iloc[-1]

    rolling_avg = data['Price_Change'].rolling(window=min(14, len(data))).mean().iloc[-1]
    
    return today_price_change, rolling_avg


def fetch_option_chain(api_key: str, secret_key: str, underlying_symbol: str):

    url = f"https://data.alpaca.markets/v1beta1/options/snapshots/{underlying_symbol}?feed=indicative&limit=100&expiration_date=2025-02-07"
    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": api_key,
        "APCA-API-SECRET-KEY": secret_key,
    }
    try:
        response = requests.get(url, headers=headers)
        data = response.json()
        snapshots = data.get('snapshots', {})
        return snapshots
    except requests.exceptions.RequestException as e:
        print(f"Error fetching option chain: {e}")
        return None

def parse_option_chain(option_chain):

    parsed_data = []
    
    for symbol, details in option_chain.items():
        expiration_start = len(symbol) - 15
        ticker = symbol[:expiration_start]
        expiration_date = f"20{symbol[expiration_start:expiration_start+2]}-{symbol[expiration_start+2:expiration_start+4]}-{symbol[expiration_start+4:expiration_start+6]}"
        option_type = "Call" if symbol[expiration_start+6] == "C" else "Put"
        strike_price = int(symbol[expiration_start+7:]) / 1000  
        
        greeks = details.get("greeks", {}) or {}
        implied_volatility = details.get("impliedVolatility", None)
        latest_quote = details.get("latestQuote", {})
        
        parsed_data.append({
            "ticker": ticker,
            "expiration_date": expiration_date,
            "option_type": option_type,
            "strike_price": strike_price,
            "delta": greeks.get("delta"),
            "gamma": greeks.get("gamma"),
            "rho": greeks.get("rho"),
            "theta": greeks.get("theta"),
            "vega": greeks.get("vega"),
            "implied_volatility": implied_volatility,
            "ask_price": latest_quote.get("ap"),
            "ask_size": latest_quote.get("as"),
            "bid_price": latest_quote.get("bp"),
            "bid_size": latest_quote.get("bs")
        })
    
    return pd.DataFrame(parsed_data)

def get_option_chain(api_key: str, secret_key: str, underlying_symbol: str):

    option_chain = fetch_option_chain(api_key, secret_key, underlying_symbol)
    if option_chain:
        return parse_option_chain(option_chain)
    else:
        return pd.DataFrame()
    

import numpy as np

def gbm(s0=100, mu=0.2, sigma=0.68, deltaT=52, dt=0.1):
    """
    Models a stock price S(t) using the Wiener process W(t) as
    `S(t) = S(0).exp{(mu-(sigma^2/2).t)+sigma.W(t)}`
    
    Arguments:
        s0: Initial stock price, default 100
        mu: 'Drift' of the stock (upwards or downwards), default 0.2
        sigma: 'Volatility' of the stock, default 0.68
        deltaT: The time period for which the future prices are computed, default 52 (as in 52 weeks)
        dt: The granularity of the time-period, default 0.1
    
    Returns:
        time_vector: array of time steps
        s: array with the simulated stock prices over the time-period deltaT
    """
    n_step = int(deltaT / dt)  # Number of time steps
    time_vector = np.linspace(0, deltaT, num=n_step)  # Time vector
    
    # Wiener process: cumulative sum of random normal increments
    random_increments = np.random.normal(0, np.sqrt(dt), size=n_step)
    weiner_process = np.cumsum(random_increments)
    
    # Stock price simulation
    stock_var = (mu - (sigma**2 / 2)) * time_vector
    s = s0 * np.exp(stock_var + sigma * weiner_process)
    
    return time_vector, s

def get_current_stock_price(symbol: str):

    url = "https://data.alpaca.markets/v2/stocks/trades/latest"

    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": api_key,
        "APCA-API-SECRET-KEY": secret_key,
    }

    params = {
        "symbols": symbol,  
        "feed": "iex" 
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  

        data = response.json()
        return data.get("trades", {}).get(symbol, {}).get("p") 

    except requests.exceptions.RequestException as e:
        print(f"Error fetching stock price: {e}")





In [35]:
# SCREENS STOCKS WITH 1-DAY CHANGE OF -2% AND ROLLING AVG OF CHANGE OVER 2 WEEKS OF > 0%
filtered_stocks = []
for index, stock in NASDAQ.iterrows():
    today_change, rolling_avg = get_price_change_and_rolling_avg(stock['Symbol'])
    if (today_change <= -2.00) and (rolling_avg > 0.00): 
        filtered_stocks.append(stock['Symbol'])
        print(f"Ticker: {stock['Symbol']}")
        print(f"Today's price change: {today_change}")
        print(f"2-week rolling average of price changes: {rolling_avg}")



Ticker: AMD
Today's price change: -2.448261523781232
2-week rolling average of price changes: 0.023662989634512917
Ticker: INTC
Today's price change: -2.8985503102038956
2-week rolling average of price changes: 0.1489689024510483
Ticker: CEG
Today's price change: -2.7775002902903756
2-week rolling average of price changes: 0.13618166180087052
Ticker: PDD
Today's price change: -3.0242615404011803
2-week rolling average of price changes: 1.2341203645359435
Ticker: FAST
Today's price change: -2.3206260539639727
2-week rolling average of price changes: 0.16643989488633457
Ticker: TTWO
Today's price change: -2.9911631796786087
2-week rolling average of price changes: 0.2805167534731803


In [57]:
option_chain = get_option_chain(api_key, secret_key, 'AAPL')

put_chain = option_chain[(option_chain['option_type'] == 'Put') & (option_chain['rho'].notna())].sort_values(by='strike_price', ascending=True)
price = get_current_stock_price('AAPL')


# run through gmb simulation to decide if profitable or not
# for index, contract in put_chain.iterrows():
#     strike_price = contract['strike_price']
#     implied_volatility = contract['implied_volatility']
#     drift = 0.05
#     prices = gbm(s0= price, mu= drift, sigma= implied_volatility )
#     plt.plot(prices)


import matplotlib.pyplot as plt

# Loop through each put option in the chain and simulate price movements
for index, contract in put_chain.iterrows():
    count = 0
    for i in range(100):
        strike_price = contract['strike_price']
        implied_volatility = contract['implied_volatility']
        
        if np.isnan(implied_volatility):  # Skip if IV is missing
            continue

        time, prices = gbm(s0=price, mu=0.025, sigma= 0.15, deltaT=2, dt=0.1)

        if prices[-1] > strike_price:
            count +=1

    print("PROFITABILITY CHANCE = ", (count / 100)*100)

    # # Create a new figure for each simulation
    # plt.figure(figsize=(8, 5))
    # plt.plot(time, prices, label="Simulated Price Path")
    # plt.axhline(y=strike_price, color='r', linestyle='--', label=f"Strike Price: {strike_price}")
    
    # plt.xlabel("Time Steps")
    # plt.ylabel("Price")
    # plt.title(f"GBM Simulation for Strike Price {strike_price}")
    # plt.legend()
    # plt.show()

put_chain

PROFITABILITY CHANCE =  92.0
PROFITABILITY CHANCE =  88.0
PROFITABILITY CHANCE =  88.0
PROFITABILITY CHANCE =  83.0
PROFITABILITY CHANCE =  91.0
PROFITABILITY CHANCE =  82.0
PROFITABILITY CHANCE =  77.0
PROFITABILITY CHANCE =  87.0
PROFITABILITY CHANCE =  84.0
PROFITABILITY CHANCE =  80.0
PROFITABILITY CHANCE =  81.0
PROFITABILITY CHANCE =  77.0
PROFITABILITY CHANCE =  75.0
PROFITABILITY CHANCE =  64.0
PROFITABILITY CHANCE =  76.0
PROFITABILITY CHANCE =  70.0
PROFITABILITY CHANCE =  64.0
PROFITABILITY CHANCE =  67.0
PROFITABILITY CHANCE =  54.0
PROFITABILITY CHANCE =  59.0
PROFITABILITY CHANCE =  64.0
PROFITABILITY CHANCE =  63.0
PROFITABILITY CHANCE =  59.0


,ticker,expiration_date,option_type,strike_price,delta,gamma,rho,theta,vega,implied_volatility,ask_price,ask_size,bid_price,bid_size
14,AAPL,2025-02-07,Put,180.0,-0.0083,0.0009,-0.0004,-0.0422,0.0073,0.8113,0.12,257,0.03,51
19,AAPL,2025-02-07,Put,185.0,-0.0055,0.0007,-0.0003,-0.0247,0.0051,0.6816,0.05,1,0.03,30
68,AAPL,2025-02-07,Put,187.5,-0.0085,0.0010,-0.0004,-0.0365,0.0075,0.6857,0.10,197,0.03,137
25,AAPL,2025-02-07,Put,190.0,-0.0090,0.0012,-0.0004,-0.0362,0.0079,0.6493,0.09,288,0.04,100
50,AAPL,2025-02-07,Put,192.5,-0.0083,0.0012,-0.0004,-0.0312,0.0073,0.6003,0.07,1,0.04,13
88,AAPL,2025-02-07,Put,195.0,-0.0061,0.0010,-0.0003,-0.0213,0.0056,0.5349,0.06,1,0.01,28
18,AAPL,2025-02-07,Put,197.5,-0.0093,0.0015,-0.0004,-0.0304,0.0081,0.5301,0.09,168,0.02,203
64,AAPL,2025-02-07,Put,200.0,-0.0145,0.0021,-0.0007,-0.0447,0.0119,0.5297,0.11,189,0.07,219
87,AAPL,2025-02-07,Put,202.5,-0.0238,0.0032,-0.0011,-0.0692,0.0182,0.5389,0.29,80,0.03,73
34,AAPL,2025-02-07,Put,205.0,-0.0159,0.0027,-0.0007,-0.0413,0.0129,0.4532,0.12,131,0.05,230
